In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw09.ipynb")

# Homework 09: Classification

**Reading**: 

* [Classification](https://www.inferentialthinking.com/chapters/17/classification.html)

Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell to load the provided tests. Each time you start your server, you will need to execute this cell again to load the tests.

This assignment is due by the deadline listed in Canvas/Gradescope. Start early so that you can come to office hours if you're stuck. Check the course website for the office hours schedule. Late work will not be accepted as per the course expectations.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. Refer to the course expectations document to learn more about how to learn cooperatively.

For all problems that you must write our explanations and sentences for, you **must** provide your answer in the designated space. Moreover, throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook! For example, if you use `max_temperature` in your answer to one question, do not reassign it later on.

In [1]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## 1. Tobacco Road Coordinates with Classification


Welcome to Homework 09! This homework is about k-Nearest Neighbors classification (kNN). This topic is covered more in-depth in one of the Final Project options. The purpose of this homework is to reinforce the basics of this method.

## Our Dearest Neighbors

Carol is trying classify students as either attendees of Duke University or UNC at Chapel Hill. To classify the students, Carol has access to the coordinates of the location they live during the school year (wow, kind of creepy Carol). First, load in the `coordinates` table.

In [2]:
# Just run this cell!
coordinates = Table.read_table('coordinates_nc.csv')
coordinates.show(5)

As usual, let's investigate our data visually before performing any kind of numerical analysis.

In [3]:
# Just run this cell!
coordinates.scatter("longitude", "latitude", group="school")

In this case, most people probably don't recognize how the latitude and longitude relate to real life, so we can use a mapping function to put these points onto a map.

In [9]:
# Just run this cell!
colors = {"Duke":"darkblue", "UNC-CH":"cornflowerblue"}
t = Table().with_columns("lat", coordinates.column(0), 
                                      "lon", coordinates.column(1), 
                                      "color", coordinates.apply(colors.get, 2)
                        )
Circle.map_table(t, radius=5, fill_opacity=1)

### Question 1

Let's begin implementing the k-Nearest Neighbors algorithm. Define the `distance` function, which takes in two arguments: an array of numerical features, and a different array of numerical features. The function should return the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between the two arrays. Euclidean distance is often referred to as the straight-line distance formula that you may have learned previously.


<!--
BEGIN QUESTION
name: q1_1
manual: false
-->

In [10]:
def distance(arr1, arr2):
    ...

# Don't change/delete the code below in this cell
distance_example = distance(make_array(1, 2, 3), make_array(4, 5, 6))
distance_example

In [ ]:
grader.check("q1_1")

### Splitting the dataset
We'll do 2 different kinds of things with the `coordinates` dataset:
1. We'll build a classifier using coordinates for which we know the associated label; this will teach it to recognize labels of similar coordinate values. This process is known as *training*.
2. We'll evaluate or *test* the accuracy of the classifier we build on data we haven't seen before.

For reasons discussed in class and the textbook, we want to use separate datasets for these two purposes.  So we split up our one dataset into two.

### Question 2

Next, let's split our dataset into a training set and a test set. Since `coordinates` has 84 rows, let's create a training set with the first 63 rows (75% of the data) and a test set with the remaining 21 rows (25% of the data). Remember that assignment to each group should be random, so you should shuffle the table **first** (save it to `shuffled_table`), then select 63 rows for the training set and 21 rows for the testing set using the `.take()` table method with `np.arange` to specify which rows go into each set.

<!--
BEGIN QUESTION
name: q1_2
manual: false
-->

In [13]:
shuffled_table = ...
train = ...
test = ...

# DON'T CHANGE THE CODE BELOW
print("Training set:\t",   train.num_rows, "examples")
train.show(5)

print("Test set:\t",       test.num_rows, "examples")
test.show(5);

In [ ]:
grader.check("q1_2")

### Question 3

Assign `features` to an array of the labels of the features from the `coordinates` table.

*Hint: which of the column labels in the `coordinates` table are the features, and which of the column labels correspond to the class we're trying to predict?*

<!--
BEGIN QUESTION
name: q1_3
manual: false
-->

In [18]:
features = ...
features

In [ ]:
grader.check("q1_3")

### Question 4

Now define the `classify` function. This function should take in a `row` from a table like `test` and classify it based on the data in `train` using the `k`-Nearest Neighbors based on the correct `features`.

*Hint: use the `row_to_array` function we defined for you to convert rows to arrays of features so that you can use the `distance` function you defined earlier.*

*Hint 2: the skeleton code we provided iterates through each row in the training set*

<!--
BEGIN QUESTION
name: q1_4
manual: false
-->

In [20]:
def row_to_array(row, features):
    arr = make_array()
    for feature in features:
        arr = np.append(arr, row.item(feature))
    return arr

def classify(row, k, train):
    test_row_features_array = row_to_array(row, features)
    distances = make_array()
    for train_row in train.rows:
        train_row_features_array = ...
        row_distance = ...
        distances = ...
    train_with_distances = ...
    nearest_neighbors = train_with_distances.sort('Distances').take(np.arange(k))
    most_common_label = ...
    ...

# The code below will attempt to classify the first row in your 
# test dataset using a 5 nearest neighbors classifier
first_test = classify(test.row(0), 5, train)
first_test

In [ ]:
grader.check("q1_4")

### Question 5

The function `three_classify` takes in a `row` from `test` as an argument and classifies the row using a 3-Nearest Neighbors classifier. We define this function so we can use the `apply()` method to quickly classify all the rows we have in the testing data set. We can then compare the prediction from the classifier to the known label for each row to get an idea of how accurate our classifier is on the test data. 

The `test_with_prediction` table is created for you. Use this table to find the `accuracy` of a 3-NN classifier on the `test` set. `accuracy` should be a proportion (not a percentage) of the schools that were correctly predicted.

<!--
BEGIN QUESTION
name: q1_5
manual: false
-->

In [23]:
def three_classify(row):
    return classify(row, 3, train)

test_with_prediction = test.with_column("prediction", test.apply(three_classify))
labels_correct = ...
accuracy = ...
accuracy

In [ ]:
grader.check("q1_5")

In [26]:
coordinates.where('school', 'UNC-CH').num_rows

<!-- BEGIN QUESTION -->

### Question 6

Suppose you work at the leasing office for an apartment building located at the GPS coordinates 35.95979476700251, -78.9870877612551 and a student is moving in. The cell below will create a table `new_student` with these coordinates as the only row.

What school would your classifier predict they attend using a 3 nearest neighbor model?

<!--
BEGIN QUESTION
name: q1_6
manual: true
-->

In [27]:
new_student = Table().with_columns('latitude', make_array(35.95979476700251), 'longitude', make_array(-78.9870877612551))
new_student

<!-- END QUESTION -->

In [29]:
# Write your code below
...

### Question 7

There are 45 rows of Duke students and 39 rows of UNC-CH students in the `coordinates` table. If we used the entire `coordinates` table as the train set, what is the smallest value of k that would ensure that a k-Nearest Neighbor classifier would always predict Duke as the school? Assign the value to `k`. The test on this question will only verify your answer is formatted correctly, it will not check it for accuracy. 

<!--
BEGIN QUESTION
name: q1_7
manual: false
-->

In [31]:
k = ...
k

In [ ]:
grader.check("q1_7")

<!-- BEGIN QUESTION -->

### Question 8

Why do we divide our data into a training and test set instead of using all the data to train the model?


<!--
BEGIN QUESTION
name: q1_8
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 9

Why do we use an odd-numbered `k` in k-NN? Explain.


<!--
BEGIN QUESTION
name: q1_9
manual: true
-->

_Type your answer here, replacing this text._

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

When done exporting, download the .zip file by finding it in the file browswer on the left side of the screen, then right-click and select **Download**. You'll submit this .zip file for the assignment in Canvas to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()